<a href="https://colab.research.google.com/github/tsbslteisrhy/Bigdata/blob/master/chatbot_create_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 필요 패키지 임포트
import codecs
import tensorflow as tf
import keras
import numpy as np
import random, sys
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from bs4 import BeautifulSoup

In [3]:
# 파일 로드 함수정의
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines:
      data = line.split(',')
      data = data[0] + ' ' + data[1]
      result.append(data)

  result = result[1:] # header 정보 제외
  return result

In [ ]:
# 데이터 로드
chat_dataset = load_data('/content/drive/My Drive/Tensorflow_works/data/ChatbotData.csv')
chat_text = ' '.join(chat_dataset)
chat_text

In [5]:
# 문자 벡터화
chars = sorted(list(set(chat_text)))
char_index = dict((c, i) for i, c in enumerate(chars)) # 문자 - index
index_char = dict((i, c) for i, c in enumerate(chars)) # index - 문자

print(char_index)
print(index_char)

{' ': 0, '!': 1, '"': 2, '%': 3, "'": 4, '-': 5, '.': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '5': 12, '6': 13, '7': 14, '8': 15, '9': 16, ';': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'L': 23, 'N': 24, 'O': 25, 'P': 26, 'S': 27, 'X': 28, '_': 29, 'a': 30, 'c': 31, 'g': 32, 'j': 33, 'k': 34, 'n': 35, 'o': 36, 's': 37, '~': 38, '…': 39, 'ㅊ': 40, 'ㅋ': 41, 'ㅎ': 42, 'ㅜ': 43, 'ㅠ': 44, '가': 45, '각': 46, '간': 47, '갇': 48, '갈': 49, '감': 50, '갑': 51, '값': 52, '갔': 53, '강': 54, '갖': 55, '같': 56, '갚': 57, '개': 58, '객': 59, '갠': 60, '갯': 61, '갱': 62, '걍': 63, '걔': 64, '거': 65, '걱': 66, '건': 67, '걷': 68, '걸': 69, '검': 70, '겁': 71, '것': 72, '겉': 73, '게': 74, '겐': 75, '겜': 76, '겟': 77, '겠': 78, '겨': 79, '격': 80, '겪': 81, '견': 82, '결': 83, '겹': 84, '겼': 85, '경': 86, '곁': 87, '계': 88, '곗': 89, '고': 90, '곡': 91, '곤': 92, '곧': 93, '골': 94, '곰': 95, '곱': 96, '곳': 97, '공': 98, '과': 99, '관': 100, '광': 101, '괘': 102, '괜': 103, '괴': 104, '교': 105, '구': 106, '국': 107, '군': 108, '굳': 109, '굴': 110,

In [8]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록
maxlen = 20
step = 3
sentences = []
next_char = []

for i in range(0, len(chat_text) - maxlen, step):
  sentences.append(chat_text[i: i + maxlen])
  next_char.append(chat_text[i + maxlen])

In [ ]:
train_x=np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
train_y=np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentences in enumerate(sentences):

  for t, char in enumerate(sentences):
    train_x[i, t, char_index[char]] = 1

  train_y[i, char_index[next_char[i]]] = 1

train_x

In [15]:
# 모델 구성
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))


In [16]:
# 모델 학습
model.fit(train_x, train_y, batch_size=128, epochs=10)

model

Epoch 1/10
917/917 [==============================] - 6s 7ms/step - loss: 3.2640
Epoch 2/10
917/917 [==============================] - 6s 7ms/step - loss: 2.7307
Epoch 3/10
917/917 [==============================] - 6s 7ms/step - loss: 2.6005
Epoch 4/10
917/917 [==============================] - 6s 7ms/step - loss: 2.5054
Epoch 5/10
917/917 [==============================] - 6s 7ms/step - loss: 2.4392
Epoch 6/10
917/917 [==============================] - 6s 7ms/step - loss: 2.3820
Epoch 7/10
917/917 [==============================] - 6s 7ms/step - loss: 2.3327
Epoch 8/10
917/917 [==============================] - 6s 7ms/step - loss: 2.2956
Epoch 9/10
917/917 [==============================] - 7s 7ms/step - loss: 2.2595
Epoch 10/10
917/917 [==============================] - 7s 7ms/step - loss: 2.2223


In [17]:
# 모델 저장
model.save('/content/drive/My Drive/Tensorflow_works/model/chatbot.model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Tensorflow_works/model/chatbot.model/assets
